In [ ]:
# nbo_features.md
txt_path = "NBO/C24_results/C24-7-1-1.txt"
node_template_path = "node_features.xlsx"


In [ ]:
# node features
import nbo_nodes_parser

In [ ]:
# input
if __name__ == "__main__":
    mol_id = "C24"
    node_xlsx = "node_features.xlsx"
    # Optional: if creating from scratch and you want to carry README from template:
    readme_template = "node_features_template.xlsx"

    # 1) q_npa
    df_npa = parse_npa_charges("C24-ChargeFlog.txt", mol_id)
    merge_into_node_excel(node_xlsx, df_npa, ["q_npa"], readme_source_xlsx=readme_template)

    # 2) val_mayer_tot
    df_mayer = parse_mayer_total_valence("C24-9-1-n.txt", mol_id)
    merge_into_node_excel(node_xlsx, df_mayer, ["val_mayer_tot"])

    # 3) pop_mull & q_mull
    df_mull = parse_mulliken_pop_and_charge("C24-7-5-4.txt", mol_id)
    merge_into_node_excel(node_xlsx, df_mull, ["pop_mull", "q_mull"])

    # 4) q_adch
    df_adch = parse_adch("C24-7-11-1.txt", mol_id)
    merge_into_node_excel(node_xlsx, df_adch, ["q_adch"])

    # 5) q_chelpg
    df_chelpg = parse_chelpg("C24_7_12.txt", mol_id)
    merge_into_node_excel(node_xlsx, df_chelpg, ["q_chelpg"])

    # 6) q_hirsh
    df_hirsh = parse_hirshfeld_7_1_1("C24-7-1-1.txt", mol_id)
    merge_into_node_excel(node_xlsx, df_hirsh, ["q_hirsh"])

    print(f"✅ Node features merged into: {node_xlsx}")


✅ Node features merged into: node_features.xlsx


In [7]:
import nbo_edges_parser

In [8]:
# edge input
if __name__ == "__main__":
    mol_id = "C24"
    edge_xlsx = "edge_features_C24.xlsx"
    readme_template = "edge_features_template.xlsx"

    # 1) Mayer bond order (absolute)
    df_mayer = parse_mayer_bond_order("C24-9-1-e.txt", mol_id)
    merge_into_edge_excel(edge_xlsx, df_mayer, ["bo_mayer_abs"], readme_source_xlsx=readme_template)

    # 2) Wiberg bond order
    df_wiberg = parse_wiberg_bond_order("C24-9-3.txt", mol_id)
    merge_into_edge_excel(edge_xlsx, df_wiberg, ["bo_wiberg"])

    # 3) Mulliken bond order
    df_mull = parse_mulliken_bond_order("C24-9-4.txt", mol_id)
    merge_into_edge_excel(edge_xlsx, df_mull, ["bo_mull"])

    print(f"✅ Edge features merged into: {edge_xlsx}")

✅ Edge features merged into: edge_features_C24.xlsx


In [ ]:
# random data
# for node
# Build a fresh consolidated node Excel from scratch:
# - C24 from text outputs (real)
# - All available .gjf among the requested set (C23, C25, C26, C29, C30, C2822R, C2822S, C2922R, C2922S,
#   H31R, H31S, H32R, H32S, H33R, H33S, H34R, H34S, Tm, Ts)
# - Sample features using C24 per-element stats, neutrality-correct charge features.
#
# Output: /mnt/data/node_features_merged.xlsx

import os, re, numpy as np, pandas as pd
from caas_jupyter_tools import display_dataframe_to_user
from functools import reduce

OUT = "/mnt/data/node_features_merged.xlsx"

REQUESTED = [
    ("C23",    "/mnt/data/C23.gjf"),
    ("C25",    "/mnt/data/C25.gjf"),
    ("C26",    "/mnt/data/C26.gjf"),
    ("C29",    "/mnt/data/C29.gjf"),
    ("C30",    "/mnt/data/C30.gjf"),
    ("C2822R", "/mnt/data/C2822R.gjf"),
    ("C2822S", "/mnt/data/C2822S.gjf"),
    ("C2922R", "/mnt/data/C2922R.gjf"),
    ("C2922S", "/mnt/data/C2922S.gjf"),
    ("H31R",   "/mnt/data/H31R.gjf"),
    ("H31S",   "/mnt/data/H31S.gjf"),
    ("H32R",   "/mnt/data/H32R.gjf"),
    ("H32S",   "/mnt/data/H32S.gjf"),
    ("H33R",   "/mnt/data/H33R.gjf"),
    ("H33S",   "/mnt/data/H33S.gjf"),
    ("H34R",   "/mnt/data/H34R.gjf"),
    ("H34S",   "/mnt/data/H34S.gjf"),
    ("Tm",     "/mnt/data/Tm.gjf"),
    ("Ts",     "/mnt/data/Ts.gjf"),
]

node_features = ["q_npa","val_mayer_tot","q_hirsh","pop_mull","q_mull","q_adch","q_chelpg"]
charge_features = ["q_npa","q_mull","q_adch","q_chelpg"]
col_order = ["mol_id","atom_idx","element","is_H"] + node_features

clip = {
    "q_npa": (-1.5, 1.5),
    "q_mull": (-1.5, 1.5),
    "q_adch": (-1.5, 1.5),
    "q_chelpg": (-1.5, 1.5),
    "val_mayer_tot": (0.0, 5.0),
    "pop_mull": (0.0, 10.0),
    "q_hirsh": (-1.5, 1.5),
}
element_defaults = {
    "H": {"val_mayer_tot": (1.0, 0.05), "pop_mull": (1.0, 0.2)},
    "C": {"val_mayer_tot": (4.0, 0.10), "pop_mull": (6.0, 0.3)},
    "N": {"val_mayer_tot": (3.0, 0.15), "pop_mull": (5.0, 0.4)},
    "O": {"val_mayer_tot": (2.0, 0.15), "pop_mull": (4.0, 0.4)},
}

# ---- C24 parsers from text ----
def parse_atom_scalar_lines(path, value_col):
    pat = re.compile(r"Atom\s+(\d+)\((\w)\s*\):\s*([+-]?\d+(?:\.\d+)?(?:[Ee][+-]?\d+)?)")
    rows = []
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        raw = f.read()
    for m in pat.finditer(raw):
        rows.append({"atom_idx": int(m.group(1))-1, "element": m.group(2), value_col: float(m.group(3))})
    df = pd.DataFrame(rows).sort_values("atom_idx").reset_index(drop=True)
    df["is_H"] = (df["element"].str.upper()=="H").astype(int)
    return df

def parse_mayer_total_valence(path):
    pat = re.compile(r"Atom\s+(\d+)\((\w)\s*\)\s*:\s*([+-]?\d+(?:\.\d+)?(?:[Ee][+-]?\d+)?)\s+([+-]?\d+(?:\.\d+)?(?:[Ee][+-]?\d+)?)")
    rows = []
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        raw = f.read()
    for m in pat.finditer(raw):
        rows.append({"atom_idx": int(m.group(1))-1, "element": m.group(2), "val_mayer_tot": float(m.group(3))})
    df = pd.DataFrame(rows).sort_values("atom_idx").reset_index(drop=True)
    df["is_H"] = (df["element"].str.upper()=="H").astype(int)
    return df

def parse_npa(path):
    pat = re.compile(r"^\s*([A-Za-z])\s+(\d+)\s+([+-]?\d+(?:\.\d+)?(?:[Ee][+-]?\d+)?)\s+", flags=re.M)
    rows = []
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        raw = f.read()
    for m in pat.finditer(raw):
        rows.append({"atom_idx": int(m.group(2))-1, "element": m.group(1), "q_npa": float(m.group(3))})
    df = pd.DataFrame(rows).sort_values("atom_idx").reset_index(drop=True)
    df["is_H"] = (df["element"].str.upper()=="H").astype(int)
    return df

def parse_mulliken_pop_charge(path):
    pat = re.compile(r"Atom\s+(\d+)\((\w)\s*\)\s+Population:\s*([+-]?\d+(?:\.\d+)?(?:[Ee][+-]?\d+)?)\s+Net\s+charge:\s*([+-]?\d+(?:\.\d+)?(?:[Ee][+-]?\d+)?)")
    rows = []
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        raw = f.read()
    for m in pat.finditer(raw):
        rows.append({"atom_idx": int(m.group(1))-1, "element": m.group(2), "pop_mull": float(m.group(3)), "q_mull": float(m.group(4))})
    df = pd.DataFrame(rows).sort_values("atom_idx").reset_index(drop=True)
    df["is_H"] = (df["element"].str.upper()=="H").astype(int)
    return df

def parse_chelpg(path):
    pat = re.compile(r"^\s*(\d+)\((\w)\s*\)\s+([+-]?\d+(?:\.\d+)?(?:[Ee][+-]?\d+)?)", flags=re.M)
    rows = []
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        raw = f.read()
    for m in pat.finditer(raw):
        rows.append({"atom_idx": int(m.group(1))-1, "element": m.group(2), "q_chelpg": float(m.group(3))})
    df = pd.DataFrame(rows).sort_values("atom_idx").reset_index(drop=True)
    df["is_H"] = (df["element"].str.upper()=="H").astype(int)
    return df

# Build C24 dataframe
c24_parts = [
    parse_npa("/mnt/data/C24-ChargeFlog.txt"),
    parse_mayer_total_valence("/mnt/data/C24-9-1-n.txt"),
    parse_atom_scalar_lines("/mnt/data/C24-7-1-1.txt","q_hirsh"),
    parse_mulliken_pop_charge("/mnt/data/C24-7-5-4.txt"),
    parse_atom_scalar_lines("/mnt/data/C24-7-11-1.txt","q_adch"),
    parse_chelpg("/mnt/data/C24_7_12.txt"),
]
c24 = reduce(lambda l,r: pd.merge(l,r,on=["atom_idx","element","is_H"], how="outer"), c24_parts)
c24.insert(0, "mol_id", "C24")

# Per-element stats from C24
stats = {}
for feat in node_features:
    for elem, grp in c24.groupby("element"):
        vals = pd.to_numeric(grp[feat], errors="coerce").dropna().values
        if len(vals)==0: continue
        mu = float(vals.mean())
        sigma = float(vals.std(ddof=1)) if len(vals)>1 else 0.05
        stats[(feat, elem)] = (mu, sigma)

def get_sampler(seed_key):
    RNG = np.random.default_rng(20251017 + (hash(seed_key) % 10000))
    def sample_feature(elem, feat):
        if (feat, elem) in stats:
            mu, sigma = stats[(feat, elem)]
        else:
            if feat == "val_mayer_tot":
                mu, sigma = element_defaults.get(elem, {"val_mayer_tot": (2.0, 0.2)})["val_mayer_tot"]
            elif feat == "pop_mull":
                mu, sigma = element_defaults.get(elem, {"pop_mull": (2.0, 0.3)})["pop_mull"]
            else:
                mu, sigma = (0.0, 0.2)
        lo, hi = clip.get(feat, (-np.inf, np.inf))
        x = RNG.normal(mu, max(sigma, 0.05))
        if np.isfinite(lo): x = max(x, lo)
        if np.isfinite(hi): x = min(x, hi)
        return x
    def enforce_neutrality(arr):
        arr = np.asarray(arr, dtype=float)
        return arr - arr.mean() if arr.size else arr
    return sample_feature, enforce_neutrality

def parse_gjf_atoms(path):
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        lines = [ln.rstrip() for ln in f]
    cm_idx = None
    for i, ln in enumerate(lines):
        if re.match(r"^\s*[-+]?\d+\s+[-+]?\d+\s*$", ln):
            cm_idx = i; break
    if cm_idx is None:
        raise RuntimeError(f"Charge/multiplicity not found in {path}")
    elems = []
    for ln in lines[cm_idx+1:]:
        if not ln.strip(): break
        toks = ln.split()
        if len(toks)>=1 and re.match(r"^[A-Za-z]{1,2}$", toks[0]):
            elems.append(toks[0])
    if not elems:
        raise RuntimeError(f"No atoms parsed from geometry in {path}")
    is_H = [1 if e.upper()=="H" else 0 for e in elems]
    return pd.DataFrame({"atom_idx": range(len(elems)), "element": elems, "is_H": is_H})

# Start with C24
nodes = c24[col_order].copy()

# Add all available requested .gjf molecules
for mid, path in REQUESTED:
    if not os.path.exists(path):
        continue
    base = parse_gjf_atoms(path)
    sample_feature, enforce_neutrality = get_sampler(mid)
    df = base.copy(); df.insert(0, "mol_id", mid)
    for feat in node_features:
        vals = [sample_feature(e, feat) for e in base["element"]]
        if feat in charge_features: vals = enforce_neutrality(vals)
        df[feat] = vals
    for c in col_order:
        if c not in df.columns: df[c] = pd.NA
    df = df[col_order]
    nodes = pd.concat([nodes[nodes["mol_id"] != mid], df], ignore_index=True)

# Save
nodes = nodes[col_order].sort_values(["mol_id","atom_idx"]).reset_index(drop=True)
with pd.ExcelWriter(OUT, engine="xlsxwriter") as w:
    nodes.to_excel(w, sheet_name="nodes", index=False)
    pd.DataFrame({"Notes":[
        "C24 real (from text). Others mocked from .gjf atom orders; C24-styled per-element sampling; "
        "charge features neutrality-corrected; seed base=20251017."
    ]}).to_excel(w, sheet_name="README", index=False)

# Preview: show first few rows for every mol present
present = nodes["mol_id"].astype(str).unique().tolist()
preview_blocks = []
for m in ["C24","C23","C25","C26","C29","C30","C2822R","C2822S","C2922R","C2922S",
          "H31R","H31S","H32R","H32S","H33R","H33S","H34R","H34S","Tm","Ts"]:
    if m in present:
        preview_blocks.append(nodes[nodes["mol_id"]==m].head(4))
preview = pd.concat(preview_blocks, ignore_index=True) if preview_blocks else nodes.head(40)
display_dataframe_to_user("node_features_merged.xlsx — all requested molecules present (preview)", preview)

OUT



In [ ]:
# for edge
# Retry building edge_features_merged.xlsx with the approach described.
import os, re, math, numpy as np, pandas as pd
from caas_jupyter_tools import display_dataframe_to_user

OUT = "/mnt/data/edge_features_merged.xlsx"

MOLS = [
    ("C24",    "/mnt/data/C24.gjf"),
    ("C23",    "/mnt/data/C23.gjf"),
    ("C25",    "/mnt/data/C25.gjf"),
    ("C26",    "/mnt/data/C26.gjf"),
    ("C29",    "/mnt/data/C29.gjf"),
    ("C30",    "/mnt/data/C30.gjf"),
    ("C2822R", "/mnt/data/C2822R.gjf"),
    ("C2822S", "/mnt/data/C2822S.gjf"),
    ("C2922R", "/mnt/data/C2922R.gjf"),
    ("C2922S", "/mnt/data/C2922S.gjf"),
    ("H31R",   "/mnt/data/H31R.gjf"),
    ("H31S",   "/mnt/data/H31S.gjf"),
    ("H32R",   "/mnt/data/H32R.gjf"),
    ("H32S",   "/mnt/data/H32S.gjf"),
    ("H33R",   "/mnt/data/H33R.gjf"),
    ("H33S",   "/mnt/data/H33S.gjf"),
    ("H34R",   "/mnt/data/H34R.gjf"),
    ("H34S",   "/mnt/data/H34S.gjf"),
    ("Tm",     "/mnt/data/Tm.gjf"),
    ("Ts",     "/mnt/data/Ts.gjf"),
]

def parse_gjf_geometry(path):
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        lines = [ln.rstrip() for ln in f]
    cm_idx = None
    for i, ln in enumerate(lines):
        if re.match(r"^\s*[-+]?\d+\s+[-+]?\d+\s*$", ln):
            cm_idx = i; break
    if cm_idx is None:
        raise RuntimeError(f"Charge/multiplicity not found in {path}")
    elems, coords = [], []
    for ln in lines[cm_idx+1:]:
        if not ln.strip(): break
        toks = ln.split()
        if len(toks)>=4 and re.match(r"^[A-Za-z]{1,2}$", toks[0]):
            elems.append(toks[0]); coords.append([float(toks[1]), float(toks[2]), float(toks[3])])
        else:
            break
    if not elems:
        raise RuntimeError(f"No atoms parsed from geometry in {path}")
    return elems, np.array(coords, dtype=float)

Rcov = {"H":0.31,"C":0.76,"N":0.71,"O":0.66,"F":0.57,"Cl":1.02,"Br":1.20,"I":1.39,"S":1.05,"P":1.07}

def bonded(e1,e2,d):
    r1 = Rcov.get(e1,0.80); r2 = Rcov.get(e2,0.80)
    scale = 1.25 if ("H" in (e1,e2)) else 1.20
    return d <= scale*(r1+r2) and d>1e-3

def build_bonds(elems, coords):
    bonds=[]; n=len(elems)
    for i in range(n):
        for j in range(i+1,n):
            d = float(np.linalg.norm(coords[i]-coords[j]))
            if bonded(elems[i], elems[j], d):
                pair = "-".join(sorted([elems[i], elems[j]]))
                bonds.append((i,j,elems[i],elems[j],pair,d))
    return bonds

# Parse C24 edge values if available
def parse_edge_txt_generic(path):
    txt = open(path,"r",encoding="utf-8",errors="ignore").read()
    rows=[]; 
    for m in re.finditer(r"^\s*(\d+)\s+(\d+)\s+([+-]?\d+(?:\.\d+)?(?:[Ee][+-]?\d+)?)\s*$", txt, flags=re.M):
        rows.append((int(m.group(1))-1,int(m.group(2))-1,float(m.group(3))))
    if not rows:
        for m in re.finditer(r"[Bb]ond\s+(\d+)\s*[-–]\s*(\d+)\s*[:=]\s*([+-]?\d+(?:\.\d+)?(?:[Ee][+-]?\d+)?)", txt):
            rows.append((int(m.group(1))-1,int(m.group(2))-1,float(m.group(3))))
    return pd.DataFrame(rows, columns=["a1","a2","value"]).drop_duplicates()

paths = {
    "bo_mayer_abs": "/mnt/data/C24-9-1-e.txt",
    "bo_wiberg":    "/mnt/data/C24-9-3.txt",
    "bo_mull":      "/mnt/data/C24-9-4.txt",
}

c24_pair_stats = {}
c24_global_stats = {}

if all(os.path.exists(p) for p in paths.values()) and os.path.exists("/mnt/data/C24.gjf"):
    c24_elems, c24_coords = parse_gjf_geometry("/mnt/data/C24.gjf")
    for feat, p in paths.items():
        df = parse_edge_txt_generic(p)
        if not df.empty:
            # per-pair
            df["pair"] = df.apply(lambda r: "-".join(sorted([c24_elems[int(r.a1)], c24_elems[int(r.a2)]])), axis=1)
            for pair, grp in df.groupby("pair"):
                vals = grp["value"].values.astype(float)
                mu = float(vals.mean()); sigma = float(vals.std(ddof=1)) if len(vals)>1 else 0.05
                c24_pair_stats[(feat,pair)] = (mu, sigma)
            # global
            vals = df["value"].values.astype(float)
            c24_global_stats[feat] = (float(vals.mean()), float(vals.std(ddof=1)) if len(vals)>1 else 0.05)

# Generic priors (fallback)
priors = {
    ("bo_mayer_abs","C-C"): (1.0, 0.15),
    ("bo_mayer_abs","C-H"): (0.95, 0.10),
    ("bo_wiberg","C-C"):    (1.05, 0.18),
    ("bo_wiberg","C-H"):    (0.98, 0.12),
    ("bo_mull","C-C"):      (0.85, 0.20),
    ("bo_mull","C-H"):      (0.80, 0.18),
}
clip = {
    "bo_mayer_abs": (0.0, 2.5),
    "bo_wiberg":    (0.0, 2.5),
    "bo_mull":      (0.0, 2.5),
}

def sample_edge_value(pair, feat, rng):
    if (feat, pair) in c24_pair_stats:
        mu, sigma = c24_pair_stats[(feat, pair)]
    elif feat in c24_global_stats:
        mu, sigma = c24_global_stats[feat]
    elif (feat, pair) in priors:
        mu, sigma = priors[(feat, pair)]
    else:
        mu, sigma = (1.0, 0.20)
    lo, hi = clip[feat]
    x = rng.normal(mu, max(sigma, 0.05))
    return min(max(x, lo), hi)

# Build edges for all molecules
rows = []
for mol_id, path in MOLS:
    if not os.path.exists(path):
        continue
    elems, coords = parse_gjf_geometry(path)
    bonds = build_bonds(elems, coords)
    rng = np.random.default_rng(20251017 + (hash(mol_id) % 10000))
    for k,(i,j,e1,e2,pair,d) in enumerate(bonds):
        rows.append({
            "mol_id": mol_id,
            "bond_idx": k,
            "a1": i, "a2": j,
            "elem1": e1, "elem2": e2, "pair": pair, "distance": d,
            "bo_mayer_abs": sample_edge_value(pair, "bo_mayer_abs", rng),
            "bo_wiberg":    sample_edge_value(pair, "bo_wiberg", rng),
            "bo_mull":      sample_edge_value(pair, "bo_mull", rng),
        })

edges = pd.DataFrame(rows).sort_values(["mol_id","bond_idx"]).reset_index(drop=True)

with pd.ExcelWriter(OUT, engine="xlsxwriter") as w:
    edges.to_excel(w, sheet_name="edges", index=False)
    pd.DataFrame({"Notes":[
        "Bonds inferred by distance using covalent radii; indices 0-based from .gjf order. "
        "Bond-order features sampled from C24 per-pair/global stats when available, else generic priors."
    ]}).to_excel(w, sheet_name="README", index=False)

# Preview a few rows per molecule
preview = pd.concat([edges[edges['mol_id']==mid].head(5) for mid,_ in MOLS if mid in edges['mol_id'].unique()], ignore_index=True)
display_dataframe_to_user("edge_features_merged.xlsx — preview", preview)

OUT
